<a href="https://colab.research.google.com/github/Anson3208/Fine-Tune-GPT/blob/main/Fine_tuning_GPT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Fine-Tuned GPT Model

The goal of this project is to build a model that can provide a detailed description of a person based on personality based on their age, gender, and traits.

The fine-tune model is to illustrate the process of fine-tuning the GPT using the Ada model, showcasing the specific steps and techniques employed to adapt the model to a particular task. 

Please note that for this demonstration, a more cost-effective model named Ada is being utilized, which may result in responses of lower quality

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 3.4 MB/s eta 0:00:00


In [ ]:
import openai

In [ ]:
from google.colab import files
uploaded_files = files.upload()
api_key = uploaded_files['api_key.txt'].decode()

Saving api_key.txt to api_key.txt


In [ ]:
openai.api_key = api_key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = api_key

# Preparation of Personality Description dataset for fine-tuning

 It creates a dataframe and generates prompts to describe a person's personality based on their age, gender, and personality traits. The code iterates through different combinations of age, gender, and personality, generating a prompt each time. It then uses the OpenAI Completion API to receive responses for each prompt. The responses are stored in the dataframe along with relevant information such as age, gender, and personality. Finally, the dataframe is saved as a CSV file named "out_openai_completion.csv".

In [ ]:
import pandas as pd

In [ ]:
l_age = ['young', 'old']
l_gender = ['man', 'woman']
l_personality = ['optimistic', 'energetic', 'introverted'] 

f_prompt = "Write a detailed description of a {age} {gender} who has {personality} personality. Write out the entire description in a maximum of 20 words in great detail:"
f_sub_prompt = "{age}, {gender}, {personality}"

df = pd.DataFrame()
for age in l_age:
 for gender in l_gender:
  for personality in l_personality:
   for i in range(3): ## 3 times each
    prompt = f_prompt.format(age=age, gender=gender, personality=personality)
    sub_prompt = f_sub_prompt.format(age=age, gender=gender, personality=personality)
    print(sub_prompt)
    
    response = openai.Completion.create(
     model="text-davinci-003",
     prompt=prompt,
     temperature=1,
     max_tokens=500,
     top_p=1,
     frequency_penalty=0,
     presence_penalty=0
    )
    
    finish_reason = response['choices'][0]['finish_reason']
    response_txt = response['choices'][0]['text']
    
    new_row = {
      'age':age, 
      'gender':gender, 
      'personality':personality, 
      'prompt':prompt, 
      'sub_prompt':sub_prompt, 
      'response_txt':response_txt, 
      'finish_reason':finish_reason}
    new_row = pd.DataFrame([new_row])
    df = pd.concat([df, new_row], axis=0, ignore_index=True)

df.to_csv("out_openai_completion.csv")

young, man, optimistic
young, man, optimistic
young, man, optimistic
young, man, energetic
young, man, energetic
young, man, energetic
young, man, introverted
young, man, introverted
young, man, introverted
young, woman, optimistic
young, woman, optimistic
young, woman, optimistic
young, woman, energetic
young, woman, energetic
young, woman, energetic
young, woman, introverted
young, woman, introverted
young, woman, introverted
old, man, optimistic
old, man, optimistic
old, man, optimistic
old, man, energetic
old, man, energetic
old, man, energetic
old, man, introverted
old, man, introverted
old, man, introverted
old, woman, optimistic
old, woman, optimistic
old, woman, optimistic
old, woman, energetic
old, woman, energetic
old, woman, energetic
old, woman, introverted
old, woman, introverted
old, woman, introverted


# Data Preparation and Fine-Tuning Initialization

This section of the code involves the preparation of data and initialization of the fine-tuning process for generating personalized personality descriptions. It includes steps such as reading the CSV file, selecting relevant columns, renaming them appropriately, and saving the prepared data to a new CSV file. The subsequent conversion of the CSV file to JSON format and the initiation of the fine-tuning process using the OpenAI API are also executed in this section.

In [ ]:
import subprocess

In [ ]:
df = pd.read_csv('out_openai_completion.csv')

In [ ]:
df['response_txt'][10].strip() #check 

'An optimistic young woman with a bubbly personality, an eternal smile, and a desire to make the world a better place.'

In [ ]:
prepared_data = df.loc[:,['sub_prompt','response_txt']]
prepared_data.rename(columns={'sub_prompt':'prompt', 'response_txt':'completion'}, inplace=True)
prepared_data.to_csv('prepared_data.csv',index=False)


## prepared_data.csv --> prepared_data_prepared.json
subprocess.run('openai tools fine_tunes.prepare_data --file prepared_data.csv --quiet'.split())

## Start fine-tuning
subprocess.run('openai api fine_tunes.create --training_file prepared_data_prepared.jsonl --model ada --suffix "Personality"'.split())

CompletedProcess(args=['openai', 'api', 'fine_tunes.create', '--training_file', 'prepared_data_prepared.jsonl', '--model', 'ada', '--suffix', '"Personality"'], returncode=0)